In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
/kaggle/input/jigsaw-agile-community-rules/train.csv
/kaggle/input/jigsaw-agile-community-rules/test.csv
/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline/config.json
/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline/tokenizer_config.json
/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline/model.safetensors
/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline/special_tokens_map.json
/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline/sentencepiece.bpe.model


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup

In [3]:
# -----------------------------
# Load and preprocess data
# -----------------------------
# Use Kaggle paths when running on Kaggle
MODEL_PATH = "/kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline"

trn = "/kaggle/input/jigsaw-agile-community-rules/train.csv"
tst = "/kaggle/input/jigsaw-agile-community-rules/test.csv"
df_trn = pd.read_csv(trn)
df_tst = pd.read_csv(tst)


def fill_empty_examples_pandas(df):
    example_cols = ['positive_example_1', 'positive_example_2', 'negative_example_1', 'negative_example_2']
    for col in example_cols:
        df[col] = df[col].fillna('').astype(str)

    df['positive_example_1'] = df['positive_example_1'].mask(df['positive_example_1'] == '', df['positive_example_2'])
    df['positive_example_2'] = df['positive_example_2'].mask(df['positive_example_2'] == '', df['positive_example_1'])

    df['negative_example_1'] = df['negative_example_1'].mask(df['negative_example_1'] == '', df['negative_example_2'])
    df['negative_example_2'] = df['negative_example_2'].mask(df['negative_example_2'] == '', df['negative_example_1'])

    return df


def getText(value):
    return str(value) if pd.notna(value) else ''


def extract_texts(row):
    return {
        "body": getText(row["body"]),
        "rule": getText(row["rule"]),
        "subreddit": getText(row["subreddit"]),
        "pos1": f"{getText(row['positive_example_1'])}",
        "pos2": f"{getText(row['positive_example_2'])}",
        "neg1": f"{getText(row['negative_example_1'])}",
        "neg2": f"{getText(row['negative_example_2'])}",
    }

df_trn = fill_empty_examples_pandas(df_trn)
df_tst = fill_empty_examples_pandas(df_tst)

df_trn["inputs"] = df_trn.apply(extract_texts, axis=1)

df_tst['text_to_classify'] = df_tst['body'].apply(getText)
df_tst["inputs"] = df_tst.apply(extract_texts, axis=1)

text_feature_cols = [
    'body',
    'rule',
    'subreddit',
    'positive_example_1',
    'positive_example_2',
    'negative_example_1',
    'negative_example_2'
]

print("--- Comprehensive NaN Inspection for All Text Feature Columns ---")

# Count NaNs for each text feature column
print("\n--- NaN Counts per Text Feature Column ---")
print(df_trn[text_feature_cols].isnull().sum())

# Analyze rows with NaNs in 'body' (most critical)
print("\n--- Analysis for 'body' column NaNs ---")
body_nan_rows = df_trn[df_trn['body'].isnull()]
if not body_nan_rows.empty:
    print(f"Number of rows with NaN in 'body': {len(body_nan_rows)}")
    print("Rule violation distribution for rows with NaN in 'body':")
    print(body_nan_rows['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in 'body' column.")

# Analyze rows with NaNs in 'rule'
print("\n--- Analysis for 'rule' column NaNs ---")
rule_nan_rows = df_trn[df_trn['rule'].isnull()]
if not rule_nan_rows.empty:
    print(f"Number of rows with NaN in 'rule': {len(rule_nan_rows)}")
    print("Rule violation distribution for rows with NaN in 'rule':")
    print(rule_nan_rows['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in 'rule' column.")

# Analyze rows with NaNs in 'subreddit'
print("\n--- Analysis for 'subreddit' column NaNs ---")
subreddit_nan_rows = df_trn[df_trn['subreddit'].isnull()]
if not subreddit_nan_rows.empty:
    print(f"Number of rows with NaN in 'subreddit': {len(subreddit_nan_rows)}")
    print("Rule violation distribution for rows with NaN in 'subreddit':")
    print(subreddit_nan_rows['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in 'subreddit' column.")

# Analyze rows where ANY of the example columns are NaN
print("\n--- Analysis for Example Columns NaNs ---")
example_only_cols = [col for col in text_feature_cols if 'example' in col]
df_any_example_nan = df_trn[df_trn[example_only_cols].isnull().any(axis=1)]
if not df_any_example_nan.empty:
    print(f"Number of rows with NaN in ANY example column: {len(df_any_example_nan)}")
    print("Rule violation distribution for rows with NaN in ANY example column:")
    print(df_any_example_nan['rule_violation'].value_counts(normalize=True))
else:
    print("No NaN values found in any example column.")

# Overall rule_violation distribution (for comparison)
print(f"\n--- Overall rule_violation distribution: ---")
print(df_trn['rule_violation'].value_counts(normalize=True))

--- Comprehensive NaN Inspection for All Text Feature Columns ---

--- NaN Counts per Text Feature Column ---
body                  0
rule                  0
subreddit             0
positive_example_1    0
positive_example_2    0
negative_example_1    0
negative_example_2    0
dtype: int64

--- Analysis for 'body' column NaNs ---
No NaN values found in 'body' column.

--- Analysis for 'rule' column NaNs ---
No NaN values found in 'rule' column.

--- Analysis for 'subreddit' column NaNs ---
No NaN values found in 'subreddit' column.

--- Analysis for Example Columns NaNs ---
No NaN values found in any example column.

--- Overall rule_violation distribution: ---
rule_violation
1    0.508132
0    0.491868
Name: proportion, dtype: float64


In [4]:
N_EPOCHS = 8
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# -----------------------------
# Dataset
# -----------------------------
class MultiInputDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128, is_test=False): # Renamed df_trn to df for generality
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        item = {}
        for field in ["text_to_classify", "rule", "subreddit"]:
            encoded = self.tokenizer(
                row[field],
                truncation=True,
                padding='max_length',
                max_length=self.max_len,
                return_tensors="pt"
            )

            for key in encoded:
                item[f"{field}_{key}"] = encoded[key].squeeze(0)
        if not self.is_test:
          item["label"] = torch.tensor(row["rule_violation"], dtype=torch.float32)
        return item

# -----------------------------
# Model
# -----------------------------
class MultiInputBERT(nn.Module):
    def __init__(self, model_name=MODEL_PATH):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Sequential(
            nn.Linear(768 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 1) # Output a single logit
        )

    def forward(self, inputs):
        cls_outputs = []
        for field in ["text_to_classify", "rule", "subreddit"]:
            out = self.bert(
                input_ids=inputs[f"{field}_input_ids"],
                attention_mask=inputs[f"{field}_attention_mask"]
            )
            cls_outputs.append(out.last_hidden_state[:, 0])  # CLS token
        x = torch.cat(cls_outputs, dim=1)
        x = self.dropout(x)
        return self.classifier(x) # Return raw logits

In [5]:
# -----------------------------
# Training and Evaluation
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

oof_preds = np.zeros(len(df_trn))
test_preds_folds = [] # This is correct

for fold, (train_idx_orig, val_idx_orig) in enumerate(skf.split(df_trn, df_trn["rule_violation"])):
    print(f"\n----- Fold {fold+1} -----")
    # train_df = df_trn.iloc[train_idx].reset_index(drop=True)

    # Create original train and validation DataFrames for this fold
    # These are the original body, rules, subreddits, and examples
    fold_train_df_orig = df_trn.iloc[train_idx_orig].reset_index(drop=True)
    fold_val_df_orig = df_trn.iloc[val_idx_orig].reset_index(drop=True)

    # Expand the tarining data for this fold
    expanded_train_data = []
    for idx, row in fold_train_df_orig.iterrows():
        rule_text = getText(row['rule'])
        subreddit_text = getText(row['subreddit'])
        # Add original body as a training sample
        expanded_train_data.append({
            'text_to_classify': getText(row['body']),
            'rule': rule_text,
            'subreddit': subreddit_text,
            'rule_violation': row['rule_violation']
        })
        # Add positive examples
        expanded_train_data.append({
            'text_to_classify': getText(row['positive_example_1']),
            'rule': rule_text,
            'subreddit': subreddit_text,
            'rule_violation': 1.0
        })
        expanded_train_data.append({
            'text_to_classify': getText(row['positive_example_2']),
            'rule': rule_text,
            'subreddit': subreddit_text,
            'rule_violation': 1.0
        })
        # Add negative examples
        expanded_train_data.append({
            'text_to_classify': getText(row['negative_example_1']),
            'rule': rule_text,
            'subreddit': subreddit_text,
            'rule_violation': 0.0
        })
        expanded_train_data.append({
            'text_to_classify': getText(row['negative_example_2']),
            'rule': rule_text,
            'subreddit': subreddit_text,
            'rule_violation': 0.0
        })

    # Create the expanded training DataFrame for this fold
    fold_train_df_expanded = pd.DataFrame(expanded_train_data)
    fold_train_df_expanded = fold_train_df_expanded[fold_train_df_expanded['text_to_classify'] != ''].reset_index(drop=True)

    # 3. Prepare the VALIDATION data for this fold (using original body)
    # Map 'body' to 'text_to_classify' for the validation set
    fold_val_df_for_model = fold_val_df_orig.copy()
    fold_val_df_for_model['text_to_classify'] = fold_val_df_for_model['body']

    # 4. Create Datasets and DataLoaders
    train_dataset = MultiInputDataset(fold_train_df_expanded, tokenizer) # Train on expanded data
    val_dataset = MultiInputDataset(fold_val_df_for_model, tokenizer) # Validate on original body

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8) # Use a consistent batch size

    test_loader = DataLoader(MultiInputDataset(df_tst, tokenizer, is_test=True), batch_size=8, shuffle=False)


    model = MultiInputBERT().to(device)
    optimizer = AdamW(model.parameters(), lr=1e-6)
    criterion = nn.BCEWithLogitsLoss()

    num_training_steps_per_fold = len(train_loader) * N_EPOCHS
    num_warmup_steps_per_fold = int(num_training_steps_per_fold * 0.05)

    # Initialize the scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps_per_fold,
        num_training_steps=num_training_steps_per_fold
    )

    # Training Loop for this fold
    best_auc = -1.0 # Track best AUC for this fold
    best_model_state = None # To save the best model for this fold

    for epoch in range(N_EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            logits = outputs.squeeze(-1)

            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

        # Eval
        model.eval()
        preds_raw, labels_all = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}"):
                inputs = {k: v.to(device) for k, v in batch.items() if k != "label"}
                labels = batch["label"].to(device)
                outputs = model(inputs)
                logits = outputs.squeeze(-1) # Squeeze to [batch_size]

                probs = torch.sigmoid(logits).detach().cpu().tolist()
                preds_raw.extend(probs)
                labels_all.extend(labels.cpu().tolist())

            # Hard labels (for classification report, optional)
            preds = [int(p > 0.5) for p in preds_raw]

        # Print metrics
        print(classification_report(labels_all, preds, digits=3, zero_division=0))

        curr_auc = roc_auc_score(labels_all, preds_raw)
        print(f"AUC Score: {curr_auc:.4f}")

        # Save the best model for this fold based on validation AUC
        if curr_auc > best_auc:
            best_auc = curr_auc
            best_model_state = model.state_dict() # Save model weights
            print(f"  -> New best Val AUC for Fold {fold+1}: {best_auc:.4f}")

    # 6. Load best model state for this fold
    model.load_state_dict(best_model_state) # Use best_model_state
    print(f"Fold {fold+1} Best Val AUC: {best_auc:.4f}")

    # Make OOF predictions for this fold's validation set
    model.eval()
    fold_val_preds_list = []
    fold_val_true_list = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Fold {fold + 1} OOF Prediction"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(device)
            outputs = model(inputs)
            logits = outputs.squeeze(-1)  # Squeeze to [batch_size]
            probs = torch.sigmoid(logits).detach().cpu().tolist()
            fold_val_preds_list.extend(probs)
            fold_val_true_list.extend(labels.cpu().tolist())

    # Sanity check: Calculate AUC for this fold's OOF predictions
    oof_fold_auc_check = roc_auc_score(fold_val_true_list, fold_val_preds_list)
    print(f"Fold {fold + 1} OOF AUC Check: {oof_fold_auc_check:.4f} (This is the true validation AUC for this fold)")

    # Assign predictions to the correct indices in the global oof_preds array
    oof_preds[val_idx_orig] = np.array(fold_val_preds_list)  # Use val_idx_orig from kf.split
    # Make predictions on the TEST set using this fold's best model
    test_fold_preds = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f"Fold {fold + 1} Test Prediction"):
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(inputs)
            logits = outputs.squeeze(-1)  # Squeeze to [batch_size]
            probs = torch.sigmoid(logits).detach().cpu().tolist()
            test_fold_preds.extend(probs)

    test_preds_folds.append(test_fold_preds)  # Store test predictions from this fold

# -----------------------------
# Final Calculation and Submission
# -----------------------------
overall_oof_auc = roc_auc_score(df_trn['rule_violation'], oof_preds)
print(f"\n--- Overall {k_folds}-Fold OOF AUC: {overall_oof_auc:.4f} ---")

# Average test predictions across all folds
final_test_predictions = np.mean(test_preds_folds, axis=0)

# Create final submission file
submission = pd.DataFrame({
    "row_id": df_tst["row_id"],
    "rule_violation": final_test_predictions
})
submission.to_csv("submission.csv", index=False) # Save with a distinct name
print("K-Fold multi-input submission.csv created successfully!")
print(submission.head(10))


----- Fold 1 -----


2025-08-06 03:23:39.238464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754450619.431608      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754450619.494440      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of XLMRobertaModel were not initialized from the model checkpoint at /kaggle/input/xlm-roberta-base-offline/xlm_roberta_base_offline and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1: 100%|██████████| 1015/1015 [06:29<00:00,  2.61it/s]


Epoch 1 Loss: 0.6855


Validating Epoch 1: 100%|██████████| 51/51 [00:05<00:00, 10.16it/s]


              precision    recall  f1-score   support

         0.0      0.925     0.430     0.587       200
         1.0      0.636     0.966     0.767       206

    accuracy                          0.702       406
   macro avg      0.780     0.698     0.677       406
weighted avg      0.778     0.702     0.678       406

AUC Score: 0.7641
  -> New best Val AUC for Fold 1: 0.7641


...

              precision    recall  f1-score   support

         0.0      0.865     0.774     0.817       199
         1.0      0.802     0.883     0.841       206

    accuracy                          0.830       405
   macro avg      0.833     0.829     0.829       405
weighted avg      0.833     0.830     0.829       405

AUC Score: 0.8411
Fold 5 Best Val AUC: 0.8551


Fold 5 OOF Prediction: 100%|██████████| 51/51 [00:04<00:00, 10.26it/s]


Fold 5 OOF AUC Check: 0.8411 (This is the true validation AUC for this fold)


Fold 5 Test Prediction: 100%|██████████| 2/2 [00:00<00:00, 14.58it/s]


--- Overall 5-Fold OOF AUC: 0.8571 ---
K-Fold multi-input submission.csv created successfully!
   row_id  rule_violation
0    2029        0.027804
1    2030        0.980489
2    2031        0.984540
3    2032        0.983370
4    2033        0.984117
5    2034        0.015482
6    2035        0.983249
7    2036        0.016281
8    2037        0.019867
9    2038        0.978963
